## Joins with `sqlalchemy`

To join to tables in `sqlalchemy`

1. Use `join` to create a `Join` object
2. Build a `select` statement from the `join` object

## Example - Reading in the Company `db`

In [1]:
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine, func
from sqlalchemy.ext.automap import automap_base

engine = create_engine("sqlite:///databases/company_2_7_2.db")

Base = automap_base()
Base.prepare(engine, reflect=True)
Dept = Base.classes.department
Empl = Base.classes.employee

In [2]:
import pandas as pd
from sqlalchemy import select as selectq
d = selectq([Dept])
pd.read_sql_query(d, con=engine)

,DeptID,DeptName
0,31,Sales
1,33,Engineering
2,34,Clerical
3,35,Marketing


In [3]:
e = selectq([Empl])
pd.read_sql_query(e, con=engine)

,DeptID,LastName,EmpID
0,31.0,Rafferty,1
1,33.0,Jones,2
2,33.0,Heisenberg,3
3,34.0,Robinson,4
4,34.0,Smith,5
5,NaN,Williams,6


## Using `sqlalchemy.join` to create a `Join`

**Syntax:** `join(left_table, right_table, onclause=left_table.column == right_table.column)`

* Defaults to an inner join
* Set `isouter=True` to get a `LEFT OUTER JOIN`
* Set `full=True` to get a `FULL OUTER JOIN`

In [4]:
from sqlalchemy import join
j = join(Empl, Dept, onclause=Empl.DeptID == Dept.DeptID)
print(j)

employee JOIN department ON employee."DeptID" = department."DeptID"


## Inspecting the joined column names

Note that the columns are renamed `tableName_columnName`.

In [5]:
j.c.keys()

['employee_DeptID',
 'employee_LastName',
 'employee_EmpID',
 'department_DeptID',
 'department_DeptName']

## Creating a `Select` expression for a `Join`

In [6]:
from sqlalchemy import func, select as selectq

stmt = selectq([j])
print(stmt)

SELECT employee."DeptID", employee."LastName", employee."EmpID", department."DeptID", department."DeptName" 
FROM employee JOIN department ON employee."DeptID" = department."DeptID"


In [7]:
pd.read_sql_query(stmt, con=engine)

,DeptID,LastName,EmpID,DeptID,DeptName
0,31.0,Rafferty,1,31,Sales
1,33.0,Jones,2,33,Engineering
2,33.0,Heisenberg,3,33,Engineering
3,34.0,Robinson,4,34,Clerical
4,34.0,Smith,5,34,Clerical


## Left Join

In [8]:
left_join = join(Empl, Dept, onclause=Empl.DeptID==Dept.DeptID, isouter=True)
left_join_stmt = selectq([left_join])
pd.read_sql_query(left_join_stmt, con=engine)

,DeptID,LastName,EmpID,DeptID,DeptName
0,31.0,Rafferty,1,31.0,Sales
1,33.0,Jones,2,33.0,Engineering
2,33.0,Heisenberg,3,33.0,Engineering
3,34.0,Robinson,4,34.0,Clerical
4,34.0,Smith,5,34.0,Clerical
5,NaN,Williams,6,NaN,None


## Right Join

To get a `RIGHT OUTER JOIN`, just switch the order and use a `LEFT OUTER JOIN`

In [9]:
right_join = join(Dept, Empl, onclause=Empl.DeptID==Dept.DeptID, isouter=True)
right_join_stmt = selectq([right_join])
pd.read_sql_query(right_join_stmt, con=engine)

,DeptID,DeptName,DeptID,LastName,EmpID
0,31,Sales,31.0,Rafferty,1.0
1,33,Engineering,33.0,Heisenberg,3.0
2,33,Engineering,33.0,Jones,2.0
3,34,Clerical,34.0,Robinson,4.0
4,34,Clerical,34.0,Smith,5.0
5,35,Marketing,NaN,None,NaN


## Full Outer Join

**Note:** `sqllite` does not support this type of join `:/`

In [10]:
full_join = join(Empl, Dept, onclause=Empl.DeptID==Dept.DeptID, full=True)
full_join_stmt = selectq([full_join])
pd.read_sql_query(full_join_stmt, con=engine)

OperationalError: (sqlite3.OperationalError) RIGHT and FULL OUTER JOINs are not currently supported [SQL: 'SELECT employee."DeptID", employee."LastName", employee."EmpID", department."DeptID", department."DeptName" \nFROM employee FULL OUTER JOIN department ON employee."DeptID" = department."DeptID"'] (Background on this error at: http://sqlalche.me/e/e3q8)

## <font color="red"> Exercise 3 </font>

Determine all the players that have hit more than 50 home runs in a season.  The final table should include the players proper name, as well as the team name.  

**Hint:** You will need join the files listed below.  To get credit for this exercise, you will need to create a database containing these three tables and use the `sqlalchemy` join methods presented above.

In [11]:
f1, f2, f3 = ("./data/baseball/core/Batting.csv", 
              "./data/baseball/core/People.csv",
              "./data/baseball/core/Teams.csv")

In [24]:
import pandas as pd
battingq = pd.read_csv(f1)
battingq.head()

,playerID,yearID,stint,teamID,lgID,G,AB,R,H,2B,...,RBI,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP
0,abercda01,1871,1,TRO,NaN,1,4,0,0,0,...,0.0,0.0,0.0,0,0.0,NaN,NaN,NaN,NaN,0.0
1,addybo01,1871,1,RC1,NaN,25,118,30,32,6,...,13.0,8.0,1.0,4,0.0,NaN,NaN,NaN,NaN,0.0
2,allisar01,1871,1,CL1,NaN,29,137,28,40,4,...,19.0,3.0,1.0,2,5.0,NaN,NaN,NaN,NaN,1.0
3,allisdo01,1871,1,WS3,NaN,27,133,28,44,10,...,27.0,1.0,1.0,0,2.0,NaN,NaN,NaN,NaN,0.0
4,ansonca01,1871,1,RC1,NaN,25,120,29,39,11,...,16.0,6.0,2.0,2,1.0,NaN,NaN,NaN,NaN,0.0


In [38]:
from dfply import *
battingq = (battingq >>
                select(X.playerID, X.yearID, X.teamID, X.HR) >>
                mutate(id = battingq.index))
battingq.head()

,playerID,yearID,teamID,HR,id
0,abercda01,1871,TRO,0,0
1,addybo01,1871,RC1,0,1
2,allisar01,1871,CL1,0,2
3,allisdo01,1871,WS3,2,3
4,ansonca01,1871,RC1,0,4


In [78]:
battingq.dtypes

playerID    object
yearID       int64
teamID      object
HR           int64
id           int64
dtype: object

In [13]:
peopleq = pd.read_csv(f2)
peopleq.head()

,playerID,birthYear,birthMonth,birthDay,birthCountry,birthState,birthCity,deathYear,deathMonth,deathDay,...,nameLast,nameGiven,weight,height,bats,throws,debut,finalGame,retroID,bbrefID
0,aardsda01,1981.0,12.0,27.0,USA,CO,Denver,NaN,NaN,NaN,...,Aardsma,David Allan,215.0,75.0,R,R,2004-04-06,2015-08-23,aardd001,aardsda01
1,aaronha01,1934.0,2.0,5.0,USA,AL,Mobile,NaN,NaN,NaN,...,Aaron,Henry Louis,180.0,72.0,R,R,1954-04-13,1976-10-03,aaroh101,aaronha01
2,aaronto01,1939.0,8.0,5.0,USA,AL,Mobile,1984.0,8.0,16.0,...,Aaron,Tommie Lee,190.0,75.0,R,R,1962-04-10,1971-09-26,aarot101,aaronto01
3,aasedo01,1954.0,9.0,8.0,USA,CA,Orange,NaN,NaN,NaN,...,Aase,Donald William,190.0,75.0,R,R,1977-07-26,1990-10-03,aased001,aasedo01
4,abadan01,1972.0,8.0,25.0,USA,FL,Palm Beach,NaN,NaN,NaN,...,Abad,Fausto Andres,184.0,73.0,L,L,2001-09-10,2006-04-13,abada001,abadan01


In [36]:
peopleq = (peopleq >>
                select(X.playerID, X.nameGiven) >>
                mutate(id = peopleq.index))
peopleq.head()

,playerID,nameGiven,id
0,aardsda01,David Allan,0
1,aaronha01,Henry Louis,1
2,aaronto01,Tommie Lee,2
3,aasedo01,Donald William,3
4,abadan01,Fausto Andres,4


In [33]:
teamsq = pd.read_csv(f3)
teamsq.head()

,yearID,lgID,teamID,franchID,divID,Rank,G,Ghome,W,L,...,DP,FP,name,park,attendance,BPF,PPF,teamIDBR,teamIDlahman45,teamIDretro
0,1871,NaN,BS1,BNA,NaN,3,31,NaN,20,10,...,24,0.834,Boston Red Stockings,South End Grounds I,NaN,103,98,BOS,BS1,BS1
1,1871,NaN,CH1,CNA,NaN,2,28,NaN,19,9,...,16,0.829,Chicago White Stockings,Union Base-Ball Grounds,NaN,104,102,CHI,CH1,CH1
2,1871,NaN,CL1,CFC,NaN,8,29,NaN,10,19,...,15,0.818,Cleveland Forest Citys,National Association Grounds,NaN,96,100,CLE,CL1,CL1
3,1871,NaN,FW1,KEK,NaN,7,19,NaN,7,12,...,8,0.803,Fort Wayne Kekiongas,Hamilton Field,NaN,101,107,KEK,FW1,FW1
4,1871,NaN,NY2,NNA,NaN,5,33,NaN,16,17,...,14,0.840,New York Mutuals,Union Grounds (Brooklyn),NaN,90,88,NYU,NY2,NY2


In [39]:
teamsq = (teamsq >>
                select( X.yearID, X.teamID, X.name) >>
                mutate(id = teamsq.index))
teamsq.head()

,yearID,teamID,name,id
0,1871,BS1,Boston Red Stockings,0
1,1871,CH1,Chicago White Stockings,1
2,1871,CL1,Cleveland Forest Citys,2
3,1871,FW1,Fort Wayne Kekiongas,3
4,1871,NY2,New York Mutuals,4


In [15]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///databases/baseball2.db', echo=False)

In [42]:
from sqlalchemy import String, Integer
sql_types = {'id': Integer, 
             'playerID': String, 
             'yearID': Integer, 
             'teamID': String, 
             'HR': Integer, 
             'nameGiven': String,  
             'name': String, 
             }

In [43]:
schema = pd.io.sql.get_schema(battingq, 'batting', keys='id', con=engine, dtype=sql_types)
schema
engine.execute(schema)

In [44]:
schema = pd.io.sql.get_schema(peopleq, 'people', keys='id', con=engine, dtype=sql_types)
schema
engine.execute(schema)

In [45]:
schema = pd.io.sql.get_schema(teamsq, 'teams', keys='id', con=engine, dtype=sql_types)
schema
engine.execute(schema)

In [47]:
battingq.to_sql('batting', 
                  con=engine, 
                  dtype=sql_types, 
                  index=False,
                  if_exists='append')

In [48]:
peopleq.to_sql('people', 
                  con=engine, 
                  dtype=sql_types, 
                  index=False,
                  if_exists='append')

In [49]:
teamsq.to_sql('teams', 
                  con=engine, 
                  dtype=sql_types, 
                  index=False,
                  if_exists='append')

In [50]:
Base = automap_base()
Base.prepare(engine, reflect=True)
batting = Base.classes.batting
people = Base.classes.people
teams = Base.classes.teams

In [55]:
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)
session = Session()

In [56]:
from sqlalchemy import inspect
insp = inspect(engine)

In [57]:
insp.get_table_names()

['batting', 'people', 'teams']

In [58]:
insp.get_columns('batting')

[{'name': 'playerID',
  'type': VARCHAR(),
  'nullable': True,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 0},
 {'name': 'yearID',
  'type': INTEGER(),
  'nullable': True,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 0},
 {'name': 'teamID',
  'type': VARCHAR(),
  'nullable': True,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 0},
 {'name': 'HR',
  'type': INTEGER(),
  'nullable': True,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 0},
 {'name': 'id',
  'type': INTEGER(),
  'nullable': False,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 1}]

In [59]:
from sqlalchemy import select
stmt = select('*').select_from(batting)

In [60]:
from more_sqlalchemy import result_dicts
session.execute(stmt).fetchmany(5) 

[('abercda01', 1871, 'TRO', 0, 0),
 ('addybo01', 1871, 'RC1', 0, 1),
 ('allisar01', 1871, 'CL1', 0, 2),
 ('allisdo01', 1871, 'WS3', 2, 3),
 ('ansonca01', 1871, 'RC1', 0, 4)]

In [64]:
import pandas as pd
from sqlalchemy import select as selectq
b = selectq([batting])
pd.read_sql_query(b, con=engine) >> head

,playerID,yearID,teamID,HR,id
0,abercda01,1871,TRO,0,0
1,addybo01,1871,RC1,0,1
2,allisar01,1871,CL1,0,2
3,allisdo01,1871,WS3,2,3
4,ansonca01,1871,RC1,0,4


In [65]:
p = selectq([people])
pd.read_sql_query(p, con=engine) >> head

,playerID,nameGiven,id
0,aardsda01,David Allan,0
1,aaronha01,Henry Louis,1
2,aaronto01,Tommie Lee,2
3,aasedo01,Donald William,3
4,abadan01,Fausto Andres,4


In [75]:
t = selectq([teams])
pd.read_sql_query(t, con=engine) >> head

,yearID,teamID,name,id
0,1871,BS1,Boston Red Stockings,0
1,1871,CH1,Chicago White Stockings,1
2,1871,CL1,Cleveland Forest Citys,2
3,1871,FW1,Fort Wayne Kekiongas,3
4,1871,NY2,New York Mutuals,4


In [106]:
from sqlalchemy import join
left_join1 = join(batting, people, onclause=batting.playerID==people.playerID, isouter=True)
left_join_stmt1 = selectq([left_join1])
batandpeople = pd.read_sql_query(left_join_stmt1, con=engine)
batandpeople >> head

,playerID,yearID,teamID,HR,id,playerID,nameGiven,id
0,abercda01,1871,TRO,0,0,abercda01,Francis Patterson,20
1,addybo01,1871,RC1,0,1,addybo01,Robert Edward,101
2,allisar01,1871,CL1,0,2,allisar01,Arthur Algernon,230
3,allisdo01,1871,WS3,2,3,allisdo01,Douglas L.,234
4,ansonca01,1871,RC1,0,4,ansonca01,Adrian Constantine,399


In [ ]:
left_join3 = join(batandpeople, teams, onclause= and_(batandpeople.teamID==teams.teamID, 
                                                      batandpeople.yearID == teams.yearID),
                  isouter=True)
left_join_stmt3 = selectq([left_join3])
batpeandteams = pd.read_sql_query(left_join_stmt3, con=engine)
batpeandteams >> head

In [104]:
from sqlalchemy import and_
left_join2 = join(batting, teams, onclause= and_(batting.teamID==teams.teamID, batting.yearID == teams.yearID),
                  isouter=True)
left_join_stmt2 = selectq([left_join2])
batandteams = pd.read_sql_query(left_join_stmt2, con=engine)
batandteams >> head

,playerID,yearID,teamID,HR,id,yearID,teamID,name,id
0,abercda01,1871,TRO,0,0,1871,TRO,Troy Haymakers,7
1,addybo01,1871,RC1,0,1,1871,RC1,Rockford Forest Citys,6
2,allisar01,1871,CL1,0,2,1871,CL1,Cleveland Forest Citys,2
3,allisdo01,1871,WS3,2,3,1871,WS3,Washington Olympics,8
4,ansonca01,1871,RC1,0,4,1871,RC1,Rockford Forest Citys,6


In [93]:
batandpeople.dtypes

playerID     object
yearID        int64
teamID       object
HR            int64
id            int64
playerID     object
nameGiven    object
id            int64
dtype: object

In [ ]:
join3 = join(batandpeople, batandteams, onclause= and_(batandpeople.playerID == batandteams.playerID, 
                                                       batandpeople.yearID == batandteams.yearID,
                                                       batandpeople.teamID == batandteams.teamID), isouter = True)
join3_stmt3 = selectq([join3])
finaldf = pd.read_sql_query(join3_stmt3, con=engine)
finaldf >> head

In [ ]:
from sqlalchemy import select as select_sql
hrfilter = (select_sql([finaldf.playerID, finaldf.teamID, finaldf.yearID, 
                        finaldf.name, fianldf.HR, finaldf.nameGiven])
            .select_from(finaldf)
            .where(finaldf.HR > 50)
           )
pd.read_sql_query(hrfilter, con=eng).head()

## Up Next

Stuff